03. DATA PREPARATION

In [1]:
import pandas as pd
import numpy as np

filepath = '../../data/coords/old10s-60f.csv'

df = pd.read_csv(filepath)

df.columns

Index(['frame_index', 'ball', 'gk0', 'gk1', 'id_1_team_1', 'id_2_team_1',
       'id_3_team_1', 'id_4_team_0', 'id_5_team_1', 'id_6_team_0',
       'id_7_team_1', 'id_8_team_1', 'id_9_team_1', 'id_10_team_0',
       'id_11_team_1', 'id_12_team_0', 'id_13_team_1', 'id_14_team_0',
       'id_15_team_0', 'id_16_team_0', 'id_17_team_1', 'id_18_team_1',
       'id_19_team_0', 'id_22_team_0', 'id_23_team_1', 'id_24_team_1',
       'id_28_team_0', 'id_29_team_1', 'id_30_team_1', 'id_34_team_1',
       'id_36_team_1', 'id_39_team_1', 'id_43_team_1', 'id_44_team_1',
       'id_53_team_0', 'id_61_team_0', 'id_82_team_1', 'id_83_team_1',
       'id_99_team_0', 'id_105_team_1', 'id_117_team_1', 'id_137_team_0',
       'id_146_team_0', 'id_147_team_1', 'id_149_team_1', 'id_151_team_0',
       'id_157_team_1', 'id_162_team_1', 'id_163_team_0', 'id_164_team_1',
       'id_165_team_0', 'id_168_team_1', 'id_169_team_1', 'id_170_team_1',
       'id_171_team_1', 'id_173_team_0', 'id_174_team_1', 'id_182_

Create good subsets of frames

In [2]:
# Exclude the frame index column (assume it's the first column)
data_cols = df.columns[1:]

# Count non-empty (non-NaN) cells per row
df['detection_count'] = df[data_cols].notnull().sum(axis=1)

# Find indices of good frames
good_mask = df['detection_count'] > 18

# Find all consecutive subsets of good frames (no min length)
subsets = []
start = None
for idx, is_good in enumerate(good_mask):
    if is_good:
        if start is None:
            start = idx
    else:
        if start is not None:
            subsets.append((start, idx))
            start = None
# Handle sequence that goes to the end
if start is not None:
    subsets.append((start, len(df)))

# Save each subset as a CSV
import os
os.makedirs('../../data/data_preparation/subsets', exist_ok=True)
for i, (start, end) in enumerate(subsets):
    if end - start >= 10:
        subset = df.iloc[start:end]
        subset.to_csv(f'../../data/data_preparation/subsets/subset{i+1}.csv', index=False)

Interpolate good subsets

In [12]:
import pandas as pd

df = pd.read_csv('../../data/train/ss1.csv')

# Convert all columns except the index (if any) to numeric, coercing errors to NaN
for col in df.columns:
    if col != 'frame':  # replace 'frame' with your index column if needed
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Interpolate all numeric columns (coordinates)
df.interpolate(method='linear', limit_direction='both', inplace=True)

# Save the interpolated DataFrame
df.to_csv('../../data/train/ss1_interpolated.csv', index=False)